# 00_quick_start

This quick-start notebook loads the sample sales dataset, shows a simple monthly sales plot, and demonstrates a minimal Prophet forecasting example (if `prophet` is installed). Use this notebook to verify the project runs locally.

In [ ]:
# 1. Imports
import os
import pandas as pd
import matplotlib.pyplot as plt

# Create outputs directory in case it does not exist
os.makedirs('outputs', exist_ok=True)

print('Libraries imported successfully')

In [ ]:
# 2. Load sample data
# The sample CSV is expected at: data/sample_sales_data.csv
try:
    df = pd.read_csv('data/sample_sales_data.csv')
    print('Loaded sample dataset — rows:', len(df))
    display(df.head())
except FileNotFoundError:
    print('Sample dataset not found. Make sure the file exists at', 'data/sample_sales_data.csv')

In [ ]:
# 3. Preprocess and aggregate monthly sales
# Convert Order Date to datetime and aggregate sales by month
if 'Order Date' in df.columns:
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
    df['YearMonth'] = df['Order Date'].dt.to_period('M').astype(str)
    monthly = df.groupby('YearMonth', as_index=False)['Sales'].sum()
    monthly['YearMonth'] = pd.to_datetime(monthly['YearMonth'])
    display(monthly.head())
else:
    print('Order Date column not present in the dataframe')

In [ ]:
# 4. Plot monthly sales (saved to outputs/monthly_sales.png)
plt.figure(figsize=(10,5))
plt.plot(monthly['YearMonth'], monthly['Sales'])
plt.title('Monthly Sales')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.tight_layout()
plt.grid(False)
plt.savefig('outputs/monthly_sales.png')
plt.show()
print('Saved plot to outputs/monthly_sales.png')

## Minimal Prophet forecast (optional)
This cell will run only if `prophet` is installed in the environment. If not installed, the cell prints instructions to install it. The example forecasts 3 future months based on the sample data.

In [ ]:
# 5. Prophet forecasting (optional)
try:
    from prophet import Prophet
    # Prepare data for Prophet
    df_prophet = df[['Order Date', 'Sales']].dropna()
    df_prophet = df_prophet.groupby(pd.Grouper(key='Order Date', freq='D')).sum().reset_index()
    df_prophet = df_prophet.rename(columns={'Order Date':'ds', 'Sales':'y'})

    # Fit a minimal Prophet model (daily data)
    m = Prophet()
    m.fit(df_prophet)

    future = m.make_future_dataframe(periods=90)  # forecast 90 days (~3 months)
    forecast = m.predict(future)

    fig = m.plot(forecast)
    fig.savefig('outputs/prophet_forecast.png')
    print('Prophet forecast created and saved to outputs/prophet_forecast.png')
except Exception as e:
    print('Prophet not available or an error occurred. To install Prophet, run:')
    print('pip install prophet')
    print('\nFull error:\n', e)